In [1]:
import argparse
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms
from tqdm.notebook import trange
import ESN
from scipy.special import softmax
from multiprocessing import Process, Manager, Pool
import matplotlib.pyplot as plt
# from torch.optim.lr_scheduler import StepLR

In [2]:
kwargs = {'num_workers': 1, 'pin_memory': True}

train_dataset = datasets.MNIST('../data', train=True, download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ]))

test_dataset = datasets.MNIST('../data', train=False, transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ]))

train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1, shuffle=True, **kwargs)

In [3]:
train_imgs = np.zeros((60000, 28*28))
train_labels = np.zeros((60000, 10))

for batch_num, (data, label) in enumerate(train_loader):
	img = torch.flatten(data)
	train_imgs[batch_num] = img
	train_labels[batch_num] = np.zeros(10)
	train_labels[batch_num][label] = 1

test_imgs = np.zeros((10000, 28*28))
test_labels = np.zeros((10000, 1))

for batch_num, (data, label) in enumerate(test_loader):
	img = torch.flatten(data)
	test_imgs[batch_num] = img
	test_labels[batch_num] = label

In [4]:
def train_reservoir(RES_SIZE, ALPHA, SPARSITY):
	Echo = ESN.ESN(inSize=28*28, outSize=10, resSize=RES_SIZE, alpha=ALPHA, sparsity=SPARSITY)
	RA_Train  = Echo.reservoir(train_imgs)
	Wout = np.dot(np.linalg.pinv(RA_Train), train_labels)

	RA_Test = Echo.reservoir(test_imgs, new_start=True)
	Yhat = np.dot(RA_Test, Wout)

	accuracy = 0
	predictions = np.zeros(10000)
	for num, line in enumerate(Yhat):
		Yhat[num] = softmax(line)
		predictions[num] = np.argmax(Yhat[num], 0)

		accuracy += predictions[num] == test_labels[num]

	return 100 * accuracy / 10000

In [5]:
result = train_reservoir(10000, 0.9, 0.25)

In [6]:
# results = []
#
# # RES_SIZE = 1000
# ALPHA = 0.9
# SPARSITY = 0.25
# for RES_SIZE in trange(1500, 5000, 100):
# 	# for ALPHA in np.arange(0.70, 0.95, 0.05):
# 	# 	for SPARSITY in np.arange(0.05, 0.25, 0.05):
# 	try:
# 		output = train_reservoir(RES_SIZE, ALPHA, SPARSITY)
# 		results.append({(RES_SIZE, ALPHA, SPARSITY) : output})
# 		print(f'{RES_SIZE} nodes: {output[0]}')
# 	except Exception as e:
# 		pass